In [1]:
from zenml import __version__
print(f"ZenML version: {__version__}")


ZenML version: 0.71.0


In [2]:
from pymongo import MongoClient

client = MongoClient("mongodb", 27017)
db = client.test_db
print(db.list_collection_names())

[]


In [16]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(host="qdrant", port=6333)
# list existing collections to test connection
collections = qdrant.get_collections()
print(collections)


collections=[CollectionDescription(name='rag_collection')]


In [15]:
import os
import requests
import json

# Get the Ollama host from environment variables
ollama_host = os.getenv("OLLAMA_HOST", "http://host.docker.internal:11434")

# Define a test payload with the model name
payload = {
    "model": "llama2",  # Replace with the name of an available model
    "prompt": "Hello, world!",
    "parameters": {"max_tokens": 50}
}
full_response = ""
try:
    # Send a POST request to the Ollama API
    with requests.post(f"{ollama_host}/api/generate", json=payload, stream=False) as response:
        if response.status_code == 200:
            print("Connection successful! Streaming response...")
            for line in response.iter_lines():
                if line:
                    data = json.loads(line.decode("utf-8"))
                    full_response += data.get("response", "")
            print("Full Response:", full_response)
        else:
            print(f"Connection failed with status code {response.status_code}")
            print("Response:", response.text)
except requests.exceptions.RequestException as e:
    print("Error connecting to Ollama:", e)


Connection successful! Streaming response...
Full Response: Hello there! It's nice to meet you. Is there something I can help you with or would you like to chat?
